# Check Constraints

Una restricción `CHECK` permite especificar una condición sobre el valor de una fila.
La sintaxis es:

``` 
CREATE TABLE table_name (
  column1 datatype [ NULL | NOT NULL ],
  column2 datatype [ NULL | NOT NULL ],

  ...

  CONSTRAINT constraint_name
    CHECK [ NOT FOR REPLICATION ] (column_name condition)

);
```
*table_name*       El nombre de la tabla a la cual le agregaremos la restricción `CHECK`.<br>
*constraint_name* El nombre que le asignaremos a la restricción. <br>
*column_name* La columna a la cual se le aplica la restricción.<br>
*condition* La condicion que se debe cumplir.



In [15]:
--Para dejar en blanco la base de datos prueba ejecute este bloque
--Ejecute este bloque(colapsado) para usarla 
USE master
IF EXISTS 
   (
     SELECT name FROM master.dbo.sysdatabases 
     WHERE name = 'Pruebas'
    )
    DROP DATABASE Pruebas

CREATE DATABASE Pruebas
GO
USE Pruebas


Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:02.215

Por ejemplo el salario de un empleado debe ser mayor a 0, por lo tanto a esa columna le ponemos esta restricción

In [16]:
CREATE TABLE Empleados
( 
    idEmpleado INT NOT NULL,
    apellido VARCHAR(50) NOT NULL,
    nombre VARCHAR(50),
    salario numeric(10,3),
    CONSTRAINT check_salario CHECK (salario > 0)
);


Commands completed successfully.

Total execution time: 00:00:00.119

¿Que pasa si agregamos a un empleado con un salario menor o igual a cero?


Respuesta: Lanza un conflicto al intentar agregar un nuevo empleado ya que se ha establecido que el valor que se le dará al atributo salario debe ser mayor de 0.

In [17]:
INSERT  empleados
    (idEmpleado,  nombre, apellido,salario)
    VALUES (1, 'Rosario', 'Uzárraga', 0 )

: Msg 547, Level 16, State 0, Line 1
The INSERT statement conflicted with the CHECK constraint "check_salario". The conflict occurred in database "Pruebas", table "dbo.Empleados", column 'salario'.

The statement has been terminated.

Total execution time: 00:00:00.215

Agregue una restricción para que la columna sexo solo permita los valores 'M' o 'F'

In [18]:
ALTER TABLE empleados ADD sexo char(1) CONSTRAINT Check_sexo CHECK (sexo='M' or sexo='F')

Commands completed successfully.

Total execution time: 00:00:00.097

In [19]:
-- Agregue una fila que cumpla la condición
INSERT  empleados
    (idEmpleado,  nombre, apellido,salario,sexo)
    VALUES (2, 'Martha', 'Medina', 100,'F')

(1 row affected)

Total execution time: 00:00:00.023

In [20]:
-- Agregue una fila que NO cumpla la condición
INSERT  empleados
    (idEmpleado,  nombre, apellido,salario,sexo)
    VALUES (2, 'Martha', 'Medina', 100,'T')

: Msg 547, Level 16, State 0, Line 2
The INSERT statement conflicted with the CHECK constraint "Check_sexo". The conflict occurred in database "Pruebas", table "dbo.Empleados", column 'sexo'.

The statement has been terminated.

Total execution time: 00:00:00.006

In [21]:
SELECT * FROM empleados

(1 row affected)

Total execution time: 00:00:01.602

idEmpleado,apellido,nombre,salario,sexo
2,Medina,Martha,100.000,F


Agregue dos columnas: `nacimiento` y `contratación` donde la fecha de contratación no debe ser anterior al nacimiento

In [22]:
ALTER TABLE empleados ADD nacimiento DATE 
ALTER TABLE empleados ADD contratacion DATE 
 
GO

ALTER TABLE empleados ADD CONSTRAINT Check_ValidaFechas CHECK (contratacion>nacimiento)

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.050

In [52]:
-- Agregue una fila que cumpla la condición
INSERT  empleados (idEmpleado,  nombre, apellido,salario,sexo,nacimiento,contratacion)
VALUES (3, 'Daniel', 'Campos', 100,'M','1998-12-14','2015-08-25')


: Msg 547, Level 16, State 0, Line 2
The INSERT statement conflicted with the CHECK constraint "Check_Fechas". The conflict occurred in database "Pruebas", table "dbo.Empleados".

The statement has been terminated.

Total execution time: 00:00:00.008

In [51]:
-- Agregue una fila que NO cumpla la condición
INSERT  empleados(idEmpleado,  nombre, apellido,salario,sexo,nacimiento,contratacion)
VALUES (3, 'Daniel', 'Campos', 100,'M','1998-12-14','1995-08-25')


: Msg 547, Level 16, State 0, Line 2
The INSERT statement conflicted with the CHECK constraint "Check_ValidaFechas". The conflict occurred in database "Pruebas", table "dbo.Empleados".

The statement has been terminated.

Total execution time: 00:00:00.010

La fecha de nacimiento del empleado no puede ser posterior al la fecha actual. Utilice la funcion del sistema `GETDATE()` para definir la condición.

In [25]:
ALTER TABLE empleados ADD CONSTRAINT Check_fechaNac CHECK (nacimiento<=GETDATE())

--DATEDIFF(Y,Fecha1,Fecha2)>=18

Commands completed successfully.

Total execution time: 00:00:00.041

In [49]:
-- Agregue una fila que cumpla la condición
INSERT  empleados
    (idEmpleado,  nombre, apellido,salario,sexo,nacimiento,contratacion)
    VALUES (5, 'Juan', 'Es', 2,'M','2000-12-23','2019-02-23')

(1 row affected)

Total execution time: 00:00:00.022

In [50]:
-- Agregue una fila que NO cumpla la condición
INSERT  empleados
    (idEmpleado,  nombre, apellido,salario,sexo,nacimiento,contratacion)
    VALUES (5, 'Juan', 'Es', 2,'M','2038-12-14',GETDATE())

: Msg 547, Level 16, State 0, Line 2
The INSERT statement conflicted with the CHECK constraint "Check_ValidaFechas". The conflict occurred in database "Pruebas", table "dbo.Empleados".

The statement has been terminated.

Total execution time: 00:00:00.019

De hecho la empresa no puede contratar a menores de edad. Redefina la condición, eliminando la anterior y creandola otra vez

In [31]:
ALTER TABLE empleados DROP CONSTRAINT Check_fechaNac;
 GO
DELETE Empleados
ALTER TABLE empleados ADD CONSTRAINT Check_Fechas CHECK((DATEDIFF(YEAR,nacimiento,contratacion)>=18))

: Msg 3728, Level 16, State 1, Line 1
'Check_fechaNac' is not a constraint.

: Msg 3727, Level 16, State 0, Line 1
Could not drop constraint. See previous errors.

(3 rows affected)

Total execution time: 00:00:00.055

In [32]:
-- Agregue una fila que cumpla la condición
INSERT INTO empleados (idEmpleado,  nombre, apellido,salario,sexo,nacimiento,contratacion) VALUES 
(25,'Daniel','Campos',3,'M','1998-12-14',GETDATE())

(1 row affected)

Total execution time: 00:00:00.023

In [47]:
-- Agregue una fila que NO cumpla la condición
INSERT INTO empleados (idEmpleado,  nombre, apellido,salario,sexo,nacimiento,contratacion) VALUES 
(62,'Daniel','Medina',60,'M','2010-10-15',GETDATE())

: Msg 547, Level 16, State 0, Line 2
The INSERT statement conflicted with the CHECK constraint "Check_Fechas". The conflict occurred in database "Pruebas", table "dbo.Empleados".

The statement has been terminated.

Total execution time: 00:00:00.010

In [60]:
INSERT INTO empleados (idEmpleado,  nombre, apellido,salario,sexo,nacimiento,contratacion) VALUES 
(12,'Ximena','Campos',500,'F','19920424',GETDATE())


INSERT INTO empleados (idEmpleado,  nombre, apellido,salario,sexo,nacimiento,contratacion) VALUES 
(62,'Alejandra','Campos',60,'F','19922812','19980214')

(1 row affected)

: Msg 241, Level 16, State 1, Line 5
Conversion failed when converting date and/or time from character string.

Total execution time: 00:00:00.169

In [61]:
SELECT * FROM empleados

(4 rows affected)

Total execution time: 00:00:00.123

idEmpleado,apellido,nombre,salario,sexo,nacimiento,contratacion
25,Campos,Daniel,3.000,M,1998-12-14,2020-02-13
5,Es,Juan,2.000,M,2000-12-23,2019-02-23
12,Campos,Ximena,500.000,F,1992-04-24,2020-02-13
12,Campos,Ximena,500.000,F,1992-04-24,2020-02-13
